# Projet DATA732
## Label Assignation with DeBerta Model
Labelisation 

In [1]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import string
from joblib import Parallel, delayed

import numpy as np
import pandas as pd

from transformers import pipeline
output_notebook()

%matplotlib inline

Loading BokehJS ...

## 1) Data

In [7]:
df = pd.read_json('./new_df/df_kws_article.json', orient='records')
print(f'Number of articles: {df.shape[0]}')

Number of articles: 965


In [8]:
df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
df.head(5)

,year,month,day,url,title,description,content,lang,kws,locations,miscellaneus,organizations,persons,domain,date
0,2019,1,2,https://french.presstv.ir/Detail/2019/01/03/58...,Des terroristes de l'Armée syrienne libre envo...,Certains membres de l’Armée syrienne libre (AS...,Depuis que les Etats-Unis ont annoncé vouloir ...,fr,"{'algérie': 10, 'afrique': 6, 'terroriste': 6,...","{'Etats-Unis': 4, 'France': 2, 'Syrie': 4, 'Al...",{},"{'Moyen-Orient': 2, 'ASL': 2, 'Daech': 2}",{},french.presstv.ir,2019-01-02
1,2019,1,3,https://french.presstv.ir/Detail/2019/01/04/58...,Zoom Afrique du 4 janvier 2019,"Au Soudan, les manifestations contre el-Béchir...",Dans ce numéro de Zoom Afrique : L’actualité e...,fr,"{'niger': 4, 'occidental': 4, 'burkina': 3, 'p...","{'Niger': 2, 'Burkina Faso': 2}",{},{},{'el-Béchir': 2},french.presstv.ir,2019-01-03
2,2019,1,3,https://french.presstv.ir/Detail/2019/01/04/58...,Les terroristes veulent déstabiliser l’Algérie,"Selon un expert algérien, certains pays arabes...",Retour à la case départ: après avoir perdu la ...,fr,"{'terroriste': 10, 'algérie': 9, 'pays': 5, 'g...","{'Syrie': 4, 'États-Unis': 3, 'Arabie saoudite...",{},{},{},french.presstv.ir,2019-01-03
3,2019,1,4,https://french.presstv.ir/Detail/2019/01/05/58...,Zoom Afrique du 5 janvier 2019,Les USA envahissent le Gabon sur le dos d’Ali ...,Dans ce numéro de Zoom Afrique : L’actualité e...,fr,"{'état': 11, 'rdc': 11, 'bénin': 10, 'france':...","{'Afrique': 4, 'Bénin': 10, 'Chine': 6, 'golfe...",{},{'Azawad': 2},"{'MSC Mandy': 3, 'Bongo': 2}",french.presstv.ir,2019-01-04
4,2019,1,5,https://french.presstv.ir/Detail/2019/01/06/58...,Zoom Afrique du 6 janvier 2019,Bientôt un assaut contre la Côte d’Ivoire ?,Dans ce numéro de Zoom Afrique : L’actualité e...,fr,"{'communauté': 9, 'pays': 7, 'burkina': 7, 'fa...","{'États-Unis': 2, 'Côte d’Ivoire': 4, 'Côte d'...",{},{},{'Manuel Chang': 3},french.presstv.ir,2019-01-05


In [9]:
numberArticles = df.shape[0]
print(f'Number of articles: {numberArticles}')

Number of articles: 965


In [10]:
none_count = df['description'].isna().sum()
print(f"number None type in description: {none_count}")
# print(f'Porcentage of None value in total: {100*none_count/df.shape[0]:.2f} %')

# df = df.dropna(subset=['description'])

number None type in description: 0


### 1.1) Text Cleaning 

In [11]:
df['CleanedData']=df['content'].str.lower()

# Remove punctuation from the 'description' column
df['CleanedData'] = df['CleanedData'].str.replace('[{}]'.format(string.punctuation), '', regex=True)
df.head(5)

,year,month,day,url,title,description,content,lang,kws,locations,miscellaneus,organizations,persons,domain,date,CleanedData
0,2019,1,2,https://french.presstv.ir/Detail/2019/01/03/58...,Des terroristes de l'Armée syrienne libre envo...,Certains membres de l’Armée syrienne libre (AS...,Depuis que les Etats-Unis ont annoncé vouloir ...,fr,"{'algérie': 10, 'afrique': 6, 'terroriste': 6,...","{'Etats-Unis': 4, 'France': 2, 'Syrie': 4, 'Al...",{},"{'Moyen-Orient': 2, 'ASL': 2, 'Daech': 2}",{},french.presstv.ir,2019-01-02,depuis que les etatsunis ont annoncé vouloir c...
1,2019,1,3,https://french.presstv.ir/Detail/2019/01/04/58...,Zoom Afrique du 4 janvier 2019,"Au Soudan, les manifestations contre el-Béchir...",Dans ce numéro de Zoom Afrique : L’actualité e...,fr,"{'niger': 4, 'occidental': 4, 'burkina': 3, 'p...","{'Niger': 2, 'Burkina Faso': 2}",{},{},{'el-Béchir': 2},french.presstv.ir,2019-01-03,dans ce numéro de zoom afrique l’actualité en...
2,2019,1,3,https://french.presstv.ir/Detail/2019/01/04/58...,Les terroristes veulent déstabiliser l’Algérie,"Selon un expert algérien, certains pays arabes...",Retour à la case départ: après avoir perdu la ...,fr,"{'terroriste': 10, 'algérie': 9, 'pays': 5, 'g...","{'Syrie': 4, 'États-Unis': 3, 'Arabie saoudite...",{},{},{},french.presstv.ir,2019-01-03,retour à la case départ après avoir perdu la g...
3,2019,1,4,https://french.presstv.ir/Detail/2019/01/05/58...,Zoom Afrique du 5 janvier 2019,Les USA envahissent le Gabon sur le dos d’Ali ...,Dans ce numéro de Zoom Afrique : L’actualité e...,fr,"{'état': 11, 'rdc': 11, 'bénin': 10, 'france':...","{'Afrique': 4, 'Bénin': 10, 'Chine': 6, 'golfe...",{},{'Azawad': 2},"{'MSC Mandy': 3, 'Bongo': 2}",french.presstv.ir,2019-01-04,dans ce numéro de zoom afrique l’actualité en...
4,2019,1,5,https://french.presstv.ir/Detail/2019/01/06/58...,Zoom Afrique du 6 janvier 2019,Bientôt un assaut contre la Côte d’Ivoire ?,Dans ce numéro de Zoom Afrique : L’actualité e...,fr,"{'communauté': 9, 'pays': 7, 'burkina': 7, 'fa...","{'États-Unis': 2, 'Côte d’Ivoire': 4, 'Côte d'...",{},{},{'Manuel Chang': 3},french.presstv.ir,2019-01-05,dans ce numéro de zoom afrique l’actualité en...


### 1.2) Text concat 

In [12]:
 # Create an empty list to store the combined texts
 combined_texts = []
 # Iterate over the rows in the DataFrame
 for index, row in df.iterrows():
     # Combine words and counts from all the columns
     all_words = {}
     for col in ['kws', 'miscellaneus', 'locations', 'organizations', 'persons']:
        for word, count in row[col].items():
             if word in all_words:
                 all_words[word] += count
             else:
                 all_words[word] = count
   
     # Generate the combined text
     combined_text = ' '.join([f'{word} ' * count for word, count in all_words.items()])
   
     # Append the combined text to the list
     combined_texts.append(combined_text)

 # Add the combined text as a new column in the DataFrame
 df['combined_text_kws'] = combined_texts

## 2) Labelisation

In [13]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDebertaV2ForSequenceClassification: ['deberta.embeddings.position_ids']
- This IS expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDebertaV2ForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaV2ForSequenceClassification for predictions without further training.
c:\Users\Carlos Andres Cortes\anaconda3\Lib\site-packages\transformers\

In [14]:
candidate_labels = ["war", "politics", "Health", "police"]

def get_labels(content: str) -> list:
    # Create the classifier object inside the function
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")
    output = classifier(content, candidate_labels, multi_label=True)
    return output['scores']

In [10]:

# sequence_to_classify = "Les parties qui s’attendaient à ce que le Hirak se transforme en un vecteur de division, de violence et d’instabilité sont diablement déçues. Plus de sept mois après le début du mouvement, l’armée algérienne a tenu sa promesse : elle a protégé le peuple tout en cherchant à satisfaire du mieux qu’elle peut ses aspirations à plus de liberté et à une répartition plus égalitaire des richesses. Le chef d’état-major, Gaid Salah, peut se targuer de n’avoir à déplorer aucun mort, ni blessé grave, ni œil crevé, ni main arrachée en sept mois de protestation. Or cette performance intrigue et met en colère des parties qui espéraient voir l’Algérie tomber dans le chaos. Ce sont les mêmes parties qui ont embrasé la Libye, qui enflamment le Mali et qui ont récemment piloté une tentative de coup d’État en Tunisie. Mais en Algérie, l’armée est bien vigilante. Une cellule terroriste vient d’être démantelée, cellule composée visiblement d’agents étrangers. Des Israéliens ? Possible... Les autorités algériennes déclarent avoir neutralisé cinq extrémistes dangereux chargés de concevoir des opérations terroristes dans le pays. L’armée algérienne a annoncé ce dimanche 14 juillet avoir arrêté cinq personnes qui envisageaient de mener des opérations terroristes contre les manifestants pacifiques. Des opérations antiterroristes menées entre le 3 et le 7 juillet dans la région de Batna (400 km au sud-est d’Alger) ont conduit à l’arrestation de cinq terroristes opérant dans cette région, a indiqué le ministère algérien de la Défense, dans un communiqué cité par la chaîne al-Mayadeen. Ces personnes planifiaient de perpétrer des attentats avec des bombes artisanales contre les manifestants pacifiques à travers différentes régions du pays, a ajouté le ministère. L’Algérie est le théâtre depuis février d’un mouvement populaire de contestation qui a poussé le président Abdelaziz Bouteflika à la démission le 2 avril. Les manifestants exigent désormais le départ de tous les dirigeants ayant eu des responsabilités durant les 20 ans de présidence de Abdelaziz Bouteflika. Pour le 21e vendredi consécutif, les Algériens sont descendus dans la rue le 12 juillet. Alors que l’élection présidentielle prévue le 4 juillet a été annulée, faute de candidats, et que le mandat du président par intérim s’achève, un pouvoir algérien opaque, aux yeux des manifestants, mais toujours aux commandes, s’efforce d’organiser une transition qui soit à son avantage. Mais les manifestations de rue se poursuivent, dans un climat de menace grandissante. Un gouvernement civil, pas un État militaire, tel était le slogan des manifestants en réponse à la déclaration du général Ahmed Gaïd Salah, le chef d’état-major des armées, qui a déclaré mardi dernier que ceux qui scandent ce slogan ont des pensées empoisonnées."
# candidate_labels = ["war", "politic" , "economic"]
# output = classifier(sequence_to_classify, candidate_labels, multi_label=True)
# rint(output['labels'])
# int(output['scores'])

In [8]:
# Define the number of parallel processes
num_processes = 3  # Adjust this based on your system's capabilities

# Use joblib to parallelize the computation
df['scores'] = Parallel(n_jobs=num_processes)(
    delayed(get_labels)(row['combined_text_kws'])
    for _, row in df.iterrows()
)

c:\Users\Carlos Andres Cortes\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
#df['scores'] = df.apply(lambda x: get_labels(x['CleanedData'], classifier, candidate_labels), axis=1)

In [ ]:
df.head(5)